In [37]:
import cv2
import pandas as pd
from typing import List, Tuple

def create_annotated_video(
    df: pd.DataFrame, 
    labels_to_show: List[float],
    input_video_path: str,
    output_video_path: str,
    confidence_threshold: float = 0.3
):
    """
    Creates an annotated video showing bounding boxes and labels for the specified `labels_to_show`.
    
    Args:
        df (pd.DataFrame): DataFrame with columns:
            - image_idx: index of the frame
            - label: class label
            - confidence: detection confidence
            - x, y: bounding box center (in some coordinate system, typically normalized)
            - w, h: bounding box width and height (in the same coordinate system)
        labels_to_show (List[float]): List of labels (classes) to display in the video.
        input_video_path (str): Path to the input video file.
        output_video_path (str): Path to the output (annotated) video file.
        background_color (tuple): BGR color for both the bounding box and text background.
                                  Default is red: (0, 0, 255).
        confidence_threshold (float): Only draw bounding boxes with confidence above this value.
    """
    
    # Initialize video capture from the input file
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Could not open video file: {input_video_path}")
        return
    
    # Read video info
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    
    # Create video writer (mp4v is common for .mp4 files)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    frame_idx = 0  # We'll match this with df['image_idx']
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # We've reached the end of the input video
        
        # Filter the DataFrame for bounding boxes in this frame:
        #  - The frame index must match
        #  - The label must be in `labels_to_show`
        #  - The confidence must be above `confidence_threshold`
        frame_df = df[
            (df['image_idx'] == frame_idx) & 
            (df['label'].isin(labels_to_show)) & 
            (df['confidence'] > confidence_threshold)
        ]
        
        # Draw the bounding boxes
        for _, row in frame_df.iterrows():
            label = row['label']
            conf  = row['confidence'] 
            
            x_center = row['x']   # e.g. normalized center
            y_center = row['y']   # e.g. normalized center
            w_norm   = row['w']   # e.g. normalized width
            h_norm   = row['h']   # e.g. normalized height
            
            # Convert from "normalized" (or custom coords) to pixel coords, if needed
            # (If your df is already in pixel coordinates, remove normalization logic.)
            x1 = int(x_center - w_norm/2)
            y1 = int(y_center - h_norm/2)
            x2 = int(x_center + w_norm/2)
            y2 = int(y_center + h_norm/2)
            
            # Draw bounding box (using `background_color`)
            if label == 2 :
                background_color = (147, 20, 255)  # Pink
                t = "Female"
            else:
                background_color = (255, 0, 0)  # Blue
                t = "Male"

            cv2.rectangle(
                frame, 
                (x1, y1), 
                (x2, y2), 
                background_color, 
                4
            )
            
            # --- Modified text label with background ---
            text = f"{t} {conf:.2f}"
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            thickness = 1
            
            # Place the label just above the bounding box
            y_label = max(0, y1 - 5)
            
            # 1) Get the text size
            (text_width, text_height), baseline = cv2.getTextSize(
                text, font, font_scale, thickness
            )
            
            # 2) Define the rectangle coordinates for the text background
            rect_x1 = x1
            rect_y1 = y_label - text_height - baseline
            rect_x2 = x1 + text_width
            rect_y2 = y_label
            
            # Clamp coords to ensure we don't go out of frame bounds
            rect_x1 = max(rect_x1, 0)
            rect_y1 = max(rect_y1, 0)
            rect_x2 = min(rect_x2, frame.shape[1])
            rect_y2 = min(rect_y2, frame.shape[0])
            
            # 3) Draw the filled rectangle for the text background
            cv2.rectangle(
                frame,
                (rect_x1, rect_y1),
                (rect_x2, rect_y2),
                background_color,
                cv2.FILLED
            )
            
            # 4) Put the text in white
            cv2.putText(
                frame,
                text,
                (rect_x1, rect_y2 - baseline),
                font,
                font_scale,
                (255, 255, 255),  # White text
                thickness,
                cv2.LINE_AA
            )
            # --- End modified text label ---
        
        # Write the annotated frame to the output video
        out.write(frame)
        frame_idx += 1
    
    cap.release()
    out.release()
    print(f"Annotated video saved as: {output_video_path}")


In [38]:
import pandas as pd

# First load the CSV file into a DataFrame
df = pd.read_csv("/home/bohbot/Evyatar/exp/Joni_showoff/nectar/predict/results.csv")

# Then call your function, passing the actual DataFrame (df)
create_annotated_video(
    df,                       # <-- pass the DataFrame, not the CSV path
    labels_to_show=[1, 2],                        # class_label
    input_video_path = "/home/bohbot/Evyatar/exp/Joni_showoff/nectar/2024-01-29_16-22-42_cam1_cropped_b.mp4", 
    output_video_path = "/home/bohbot/Evyatar/exp/Joni_showoff/nectar/joni.mp4"
)


Annotated video saved as: /home/bohbot/Evyatar/exp/Joni_showoff/nectar/joni.mp4
